### 导入数据

In [59]:
from utils.dataLoader import Datasets
import numpy
import pandas

datasets = Datasets("/home/zlj/Excavator_ReID/Trained_ReID_features/1716801040.6510997_bank.npy")
bank_x = datasets.data
bank_y = datasets.target
bank_y = [f'{x}'+'bank' for x in bank_y]
bank_y = numpy.array(bank_y)
bank_label = datasets.classes
bank_label = [f'{x}'+'bank' for x in bank_label]
bank_label = numpy.array(bank_label)
bank_label_df = pandas.DataFrame(bank_label)
# print(len(bank_y))
bank_x_df = pandas.DataFrame(bank_x)
bank_y_df = pandas.DataFrame(bank_y)
data_show = pandas.concat([bank_y_df, bank_x_df], axis=1)
data_show

In [60]:
bank_label_df.T

In [61]:
from utils.dataLoader import FullDatasets
import numpy
import pandas

datasets = FullDatasets("/home/zlj/Excavator_ReID/Trained_ReID_features/1716801040.6510997_map.npy")
full_x = datasets.data
full_y = datasets.target
full_y = [f'{x}' for x in datasets.target]
full_y = numpy.array(full_y)
full_label = datasets.classes
full_label = [f'{x}' for x in datasets.classes]
full_label = numpy.array(full_label)
full_x_df = pandas.DataFrame(full_x)
full_y_df = pandas.DataFrame(full_y)
data_show = pandas.concat([full_y_df, full_x_df], axis=1)
data_show

In [62]:
full_label

In [63]:
import pandas as pd
X = numpy.concatenate((full_x, bank_x), axis=0)
y = numpy.concatenate((full_y, bank_y), axis=0)
show_y = pd.DataFrame(set(y)).T
show_y

## t-SNE or UMAP

In [64]:
from sklearn.manifold import TSNE
from umap import UMAP
# D_reduce = TSNE(n_components=2, random_state=1)
D_reduce = UMAP(n_components=2, random_state=3)

X_dd = D_reduce.fit_transform(X)

In [65]:
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

filtr_bank, filtr_full= numpy.isin(y, bank_y), numpy.isin(y, full_y) 
filtrs_bank = [y == y_l for y_l in bank_label]
filtrs_full = [y == y_l for y_l in full_label]


colormap = get_cmap('plasma')
# print(filtrs[1].all() == False)
# ax[0].scatter(X_dd[filtr_full, 0], X_dd[filtr_full, 1], color='blue')
# ax[0].scatter(X_dd[filtr_bank, 0], X_dd[filtr_bank, 1], color='red')


In [66]:
bank_label

In [67]:
full_label

### ID 1

In [68]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[0]
filtr_f = filtrs_full[0]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [69]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [70]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [71]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 2

In [72]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[1]
filtr_f = filtrs_full[1]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [73]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [74]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [75]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 3

In [76]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[2]
filtr_f = filtrs_full[2]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [77]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [78]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [79]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 4

In [80]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[3]
filtr_f = filtrs_full[3]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [81]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [82]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [83]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 5

In [84]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[4]
filtr_f = filtrs_full[4]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [86]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [87]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [88]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 6

In [89]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[5]
filtr_f = filtrs_full[5]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [90]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [91]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [92]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 7

In [149]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[6]
filtr_f = filtrs_full[6]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_b].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [94]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [95]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [96]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 8

In [148]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[7]
filtr_f = filtrs_full[7]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_b].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [103]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [104]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [105]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 9

In [147]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[8]
filtr_f = filtrs_full[8]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_b].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [118]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [119]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [120]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 10

In [146]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[9]
filtr_f = filtrs_full[9]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_b].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [133]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [134]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [135]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 11

In [145]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[10]
filtr_f = filtrs_full[10]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_b].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [137]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [138]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [139]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 12

In [144]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[11]
filtr_f = filtrs_full[11]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_b].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [141]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [142]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [143]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 13

In [151]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[12]
filtr_f = filtrs_full[12]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [152]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [153]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [154]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 14

In [156]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[13]
filtr_f = filtrs_full[13]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [157]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [158]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [159]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 15

In [160]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[14]
filtr_f = filtrs_full[14]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [161]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [162]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [163]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 16

In [164]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[15]
filtr_f = filtrs_full[15]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [166]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [167]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [168]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 17

In [169]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[16]
filtr_f = filtrs_full[16]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [170]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [172]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [173]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 18

In [174]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[17]
filtr_f = filtrs_full[17]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [175]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [177]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [178]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 19

In [179]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[18]
filtr_f = filtrs_full[18]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [180]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [181]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [182]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]

### ID 20

In [183]:
fig, ax = plt.subplots(1, 2, figsize=(21, 10))

filtr_b = filtrs_bank[19]
filtr_f = filtrs_full[19]

ax[0].scatter(X_dd[filtr_f, 0], X_dd[filtr_f, 1], color='blue', s=5)
ax[0].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1], color='red', s=5)

print(X_dd[filtr_f].shape)

for x, color in zip(X_dd[filtr_b], colormap(numpy.linspace(0, 1,len(X_dd[filtr_b])))):
    ax[1].scatter(x[0], x[1], color=color, s=5)

sc = ax[1].scatter(X_dd[filtr_b, 0], X_dd[filtr_b, 1],
                   c=colormap(numpy.linspace(0, 1, len(X_dd[filtr_b]))),
                   cmap=colormap, s=5)
cbar = fig.colorbar(sc, ax=ax[1], orientation='vertical', fraction=0.02, pad=0.001)
cbar.set_label('Time')
 
ax[0].set_xlabel('t-SNE1')
ax[0].set_ylabel('t-SNE2')
for i in range(2):
    # ax[i].set_xlim(-5, 20)
    # ax[i].set_ylim(-5, 20)
    ax[i].legend()

plt.title("t-SNE of feature bank")
plt.show()

#### 计算凸包

In [184]:
# 聚类
from sklearn.cluster import KMeans, OPTICS, HDBSCAN, DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering

cluster1 = DBSCAN()
cluster2 = DBSCAN(eps=1.5)
f_l = cluster1.fit_predict(X_dd[filtr_f])
b_l = cluster2.fit_predict(X_dd[filtr_b])

clus = pd.concat([pd.DataFrame(set(f_l)), pd.DataFrame(set(b_l))], axis=1)
clus.T


In [185]:
from scipy.spatial import ConvexHull
f_clus = []
b_clus = []

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for clu_idx in set(f_l):
    f_clu = X_dd[filtr_f][f_l==clu_idx]
    ax[0].scatter(f_clu[:, 0], f_clu[:, 1])
    if clu_idx != -1:
        f_hull = ConvexHull(f_clu) 
        ax[0].plot(f_clu[f_hull.vertices, 0], f_clu[f_hull.vertices, 1], 'r--')
        f_clus.append(f_clu[f_hull.vertices])

    
for clu_idx in set(b_l):
    b_clu = X_dd[filtr_b][b_l==clu_idx]
    ax[1].scatter(b_clu[:, 0], b_clu[:, 1])
    if clu_idx != -1:
        b_hull = ConvexHull(b_clu)
        ax[1].plot(b_clu[b_hull.vertices, 0], b_clu[b_hull.vertices, 1], 'r--')
        b_clus.append(b_clu[b_hull.vertices])
    
plt.show()

In [186]:
from utils.hull_iou import calculate_group_iou
print(calculate_group_iou(f_clus, b_clus))
print(len(f_clus), len(b_clus))
print(len(X_dd[filtr_f]), len(X_dd[filtr_b]))
# f_clu[f_hull.vertices]